# Генерация текста на основе “Алисы в стране чудес”



# Цель
Рекуррентные нейронные сети также могут быть использованы в качестве генеративных моделей.
Это означает, что в дополнение к тому, что они используются для прогнозных моделей (создания прогнозов), они могут изучать последовательности проблемы, а затем генерировать совершенно новые вероятные последовательности для проблемной
области.
Подобные генеративные модели полезны не только для изучения того, насколько хорошо модель выявила проблему, но и для того, чтобы узнать больше о самой проблемной области.

# Задачи

* Ознакомиться с генерацией текста
* Ознакомиться с системой Callback в Keras

# Выполнение работы


### Подключение модулей

In [1]:
import sys
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM


Добавим систему Callback

In [2]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print("End epoch {} of training".format(epoch))
        prnt_txt(self.model)

Функция печати сгенерированных символов

In [3]:
def prnt_txt(model):
    # pick a random seed
    start = np.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    print ("Seed:")
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    
    # generate characters
    for i in range(1000):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    print ("\nDone.")

Затем нам нужно загрузить текст ASCII для книги в память и преобразовать все символы в нижний регистр, чтобы уменьшить словарный запас, который должна выучить сеть.

In [4]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Преобразуем символы в числа

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

Суммируем набор данных

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144430
Total Vocab:  45


Разделяя книгу на последовательности, конвертируем символы в целые числа,
используя нашу таблицу поиска, которую подготовили ранее.

In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144330


Преобразуем выходные шаблоны в одну кодировку

In [8]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Создадим модель

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Определим чекпоинты для того чтобы модель работала быстрей.

In [10]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss',
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
callbacks=[MyCallback()]
callbacks.append(checkpoint)

Обучим модель использую встроеный колбек

In [12]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)

Epoch 1/20
1128/1128 [==============================] - ETA: 0s - loss: 2.5208End epoch 0 of training
Seed:
" othing to what i could say if i chose,' the duchess replied, in
a pleased tone.

'pray don't trouble "
 to the wooe the cane to the cane to the cane to the care to the care to the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the care the

1128/1128 [==============================] - 154s 137ms/step - loss: 1.9389
Epoch 7/20
1127/1128 [============================>.] - ETA: 0s - loss: 1.8800End epoch 6 of training
Seed:
" t of his
pocket, and was looking at it uneasily, shaking it every now and then,
and holding it to hi "
s hard and the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court of the court o

fpced of the thoes tay oo the was oot of the was oot of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of the was out of

1127/1128 [============================>.] - ETA: 0s - loss: 1.5442End epoch 17 of training
Seed:
" rm.

'a fine day, your majesty!' the duchess began in a low, weak voice.

'now, i give you fair warn "
ed the boutt,' said the king, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was not a little boutte of the court, and the hatter was see the court, and she was n

Сгенерируем текст на основе последней контрольной точки.

In [14]:
filename = "weights-improvement-20-1.5011.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
prnt_txt(model)

Seed:
" d she's
such a capital one for catching mice--oh, i beg your pardon!' cried
alice again, for this ti "
me they were tery like the door and taid to alice, and said to alice to herself 'the sueen side of the mors, and the morse was a little bottle would be a long say that the was anlther tore, and the cook was sert the white rabbit into the door and taid to alice, and said to alice to herself 'the sueen side of the mors, and the morse was a little bottle would be a long say that the was anlther tore, and the cook was sert the white rabbit into the door and taid to alice, and said to alice to herself 'the sueen side of the mors, and the morse was a little bottle would be a long say that the was anlther tore, and the cook was sert the white rabbit into the door and taid to alice, and said to alice to herself 'the sueen side of the mors, and the morse was a little bottle would be a long say that the was anlther tore, and the cook was sert the white rabbit into the door and taid to a